In [2]:
from akashiwo_data_scrape import *
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
# speacies id
spicies_ids = [45, 3]
spicies_names = ["Diaton.csv", "shtonela.csv"]

for id, names in zip(spicies_ids, spicies_names):
    pull_data(id, "2011/5/01", "2023/10/11", names)

In [96]:
colum_names = ['lat', 'lng', 'speciesValueAM', 'speciesValuePM', 
               'saisuiValueAM', 'saisuiValuePM', 'speciesNameKana', 
               'maxvalue', 'speciesId', 'icon_size']

for name in spicies_names:
    remove_colum(colum_names, name)

In [4]:
data = pd.read_csv("shtonela.csv")
pointIds = data["pointId"]
gatherYMDs = data["gatherYMD"]

In [11]:
main_data = {}
coordinate_data = {}
headers_to_skip_main = ["確定値／速報値", "事業・調査名"]
i = 0
for pointId, gatherYMD in tqdm(zip(pointIds, gatherYMDs), desc="scraping", unit="requests"):
    html_data = requests.get(f"https://akashiwo.jp/private/akashiwoListInit.php?qpoint_id={str(pointId)}&qspecies_id=3&qgather_ymd_s=&qgather_ymd_e={str(gatherYMD)}")
    html_data.encoding = 'utf-8'
    soup = BeautifulSoup(html_data.text, 'html.parser')
    tables = soup.find_all('table')
    times_to_duplicate = parse_main_table(tables[1], headers_to_skip_main, main_data)
    parse_coordinate_table(tables[0], coordinate_data, times_to_duplicate)
    if(i == 20): break
    i += 1

df = pd.DataFrame(main_data)
df.to_csv("hehe.csv", index=False)

scraping: 20requests [00:06,  2.96requests/s]


In [ ]:
merged_data = {**coordinate_data, **main_data}
df = pd.DataFrame(merged_data)
df.to_csv("hehe.csv", index=False)

In [5]:
def fill(data):
    if "hhh" in data:
        data["hhh"].extend([5, 6, 7])
    else:
        data["hhh"] = [1, 2, 3]
d = {}
d["hhh"] = [1, 2, 3]

fill(d)

d

{'hhh': [1, 2, 3, 5, 6, 7]}